# 2110443 - Computer Vision (2018/1)
## Lab 5 - Simple Object Recognition
In this lab, we will learn to use useful  handcrafted features to recognize objects in the provied images. This notebook includes both coding and written questions. Please hand in this notebook file with all outputs and your answer

<b>Collaboration is encouraged in this course.</b> You must turn in <b>your own write ups</b> of all problems. If you collaborate with others, you must put the names and ids of the students you worked with in below block.

Collaboration List:
- ...
- ...

In [ ]:
import cv2
import math
import time
import numpy as np
import random as rng
from matplotlib import pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display, HTML,clear_output
%matplotlib inline

## Assignment 1 - A Letter from Nowhere (Special Assignment)
We got multiple mysterious letters from nowhere. Here is one of them. As a computer engineering student, can you decipher this letter before unthinkable things happen? Your task is to write a program to <b>automate deciphering this image.</b> We hope that everything is going well after you decipher this secret. Please hand in the <b>image recognition output (put the recognition result on image)</b> and decoded message. 

<b>Basic Guidance:<b>
1. Extract each digit from image & filter out the outlier.
2. Group the recognized results into digit groups.
3. Use <b>your own designed feature</b> to classify between 0 and 1 
4. Write a function to decrypt them
 
<b>Hints:</b>
- You can use knowledge from previous lecture to improve the segmentation result by apply <b>?</b> on input image
- Specific threshold based on shape properties or simple morphological operations can be use to keep only potential contours
- This <a href="https://docs.opencv.org/3.4.2/d7/d4d/tutorial_py_thresholding.html">page</a> <b>may</b> contains useful information!
- What is the different feature between non-overlaped and overlapped stroke?

In [ ]:
### Describe how your algorithm work here (Thai or English).
'''
1. ..
2. ..
..
'''

In [ ]:
### Letter Image in "assets/Lab5-LetterFromNowhere.jpg" ###
### FILL HERE ###

## Maker Based Segmentation

In this section, we will learn how to apply a classic segmentation algorithm named <b>watershed</b>. This algorithm can be used to detect and extract objects in images that are <b>touching and/or overlapping</b> (like RBC in previous Lab!).

Assume that we want to build an application to count coins value from the following image. First of all, we need to segment each coins in to a individual connected component.

In [ ]:
sampleCoinImage = cv2.imread("assets/Lab5-SampleCoins.jpg")
sampleCoinGray = cv2.cvtColor(sampleCoinImage,cv2.COLOR_BGR2GRAY)
tempImage = cv2.cvtColor(sampleCoinImage,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,10))
plt.title("Thai Coins")
plt.imshow(tempImage)
plt.show()

The following blocks try to use edge information from previous lecture to seperate coins image into individual coin.
Standard Sobel operator is used to extract edge information and only strong edge pixels are kept based on defined threshold. Try to adjust the edge threshold value to seperate each coin from others

In [ ]:
def extractSobelEdge(thresholdEdgeVal):
    sobelX = np.uint8(np.absolute(cv2.Sobel(sampleCoinGray,cv2.CV_64F,1,0,ksize=3)))
    sobelY = np.uint8(np.absolute(cv2.Sobel(sampleCoinGray,cv2.CV_64F,0,1,ksize=3)))
    sobelXY = (sobelX + sobelY) > thresholdEdgeVal
    plt.figure(figsize=(10,10))
    plt.title("Edge Image")
    plt.imshow(sobelXY,cmap='gray')
    plt.show()
interact(extractSobelEdge, thresholdEdgeVal=widgets.IntSlider(min=0,max=255,step=1,value=1));

You can see from the above threshold edge map, even we selects a precise edge threshold, we cannot segment connected coins into individual one. In order to use watershed algorithm, markers muse be place at the corresponded location of the objects in our image. The markers can be either manual define or calculate from various image processing techniques. We will start from using <a href="https://docs.opencv.org/3.4.2/d7/d4d/tutorial_py_thresholding.html">automatic Otsu thesholding</a>.

In [ ]:
sampleCoinGray = cv2.blur(sampleCoinGray,(15,15))
_,thresholdCoinImage = cv2.threshold(sampleCoinGray,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
plt.imshow(thresholdCoinImage, cmap='gray')
plt.show()

We then apply <a href="https://docs.opencv.org/3.4.2/d7/d1b/group__imgproc__misc.html#ga8a0b7fdfcb7a13dde018988ba3a43042">distance transform</a> to the thresholding output. Distance transform calculates the approximate or precise distance from every binary image pixel to the nearest zero pixel

In [ ]:
distanceTransformOutput = cv2.distanceTransform(thresholdCoinImage,cv2.DIST_L2,3)
plt.imshow(distanceTransformOutput,cmap='gray')
plt.show()

Pick only pixel which is greater than 45% of max distance as potential coin markers.

In [ ]:
maxDistance = np.max(distanceTransformOutput)
print('Max Distance:',maxDistance)
roughMarkerPixel = np.uint8(distanceTransformOutput > 0.45 * maxDistance)
backgroundMask = np.invert(roughMarkerPixel)
plt.title('Potential Coins Location')
plt.imshow(roughMarkerPixel, cmap='gray')
plt.show()

plt.figure()
plt.title('Background Pixel')
plt.imshow(backgroundMask, cmap='gray')
plt.show()

Labeling each connected component with its unique label number by using <a href="https://docs.opencv.org/3.4.2/d3/dc0/group__imgproc__shape.html#gaedef8c7340499ca391d459122e51bef5">cv2.connectedComponents</a>.

In [ ]:
_, markers = cv2.connectedComponents(roughMarkerPixel)
# Add 1 to make the marker label start with 1
markers = markers+1

# Mark background pixel with 0
markers[np.where(backgroundMask==255)] = 0
print('Min Label:',np.min(markers),'Max Label:',np.max(markers))

plt.imshow(markers,cmap='rainbow')
plt.show()

Apply <a href="https://docs.opencv.org/3.4.2/d7/d1b/group__imgproc__misc.html#ga3267243e4d3f95165d55a618c65ac6e1">watershed algorithm</a> using created marker and input coins image. Is there any missing coin? Why? Fix the bug and state your instruction in below block.

In [ ]:
'''
How to find the missing coin?

'''

In [ ]:
outputImage = sampleCoinImage.copy()
borderImage = sampleCoinImage.copy()
markers = cv2.watershed(outputImage,markers)

# -1 is border pixel  
print('Min Label:',np.min(markers),'Max Label:',np.max(markers))

# 1 is background 
for outputMarkerIdx in range(2,np.max(markers+1)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    outputImage[np.where(markers == outputMarkerIdx)] = color
outputImage = cv2.cvtColor(outputImage,cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10,10))
plt.title('Segmentation Result')
plt.imshow(outputImage)
plt.show()

borderImage[np.where(markers == -1)] = (0,255,0)
borderImage = cv2.cvtColor(borderImage,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,10))
plt.title('Segmentation Boundary')
plt.imshow(borderImage)
plt.show()

### Assignment 2 - Coin Counter

From the above section, we can gracefully segment overlpping coins! Your today task is to implement a program to count total coin value from test images provided in assets folder. Please include two of your own test images by using smartphone camera. The images should show robustness of your designed alogorithm such as overlapping coins, tilt camera angle or shadow handling. (Optional) You will get extra points if you can use <b>same parameters</b> for all test images. Don't forget to show your work in step in below block.

<b>Remark : </b>
- There are eight test images in assets folder. Please hand in the recognition result in image files. 
- You must not count Yen Coin!!!

<b>Basic Guidance:<b>
1. Segment each coin into individual connected component and find the bounding box which can enclose those connect components by <a href="https://docs.opencv.org/3.4.2/d3/dc0/group__imgproc__shape.html#gacb413ddce8e48ff3ca61ed7cf626a366">cv2.boundingRect</a>
2. For each component, extract <b>useful features</b> (you have to design this by youself).
3. Classify those components into Thai Coin classes.  

<b>Hints:</b>
- How do to discard noise/fill small hole from segmentation mask output?
- From the beginning of this class, you had learned many potential image features, such as edge, color, contour and shape. Use them wisely.
- Internet is your friend. You can search for relavent research papers and use their algorithm, but you must <b>give proper credits</b> by citing them in this notebook. 

In [ ]:
### Describe how your algorithm work here (Thai or English). You can provide any visualization if you want.
'''
1. ..
2. ..
..
'''

In [ ]:
### FILL HERE ###